In [2]:
from finn.core.modelwrapper import ModelWrapper

model = ModelWrapper("/workspace/brevitas/tests/brevitas_finn/brevitas/debug.onnx")

node0 = model.graph.node[0]

print(node0)

input: "inp.1"
output: "808"
op_type: "DebugMarker"
attribute {
  name: "export_debug_name"
  s: "encoder.encoder.0.mconv.0.input"
  type: STRING
}
domain: "finn.custom_op.general"



In [2]:
import torch
import numpy as np

from finn.core.modelwrapper import ModelWrapper
import finn.core.onnx_exec as oxe
from brevitas.export import export_finn_onnx, enable_debug
#from brevitas_examples.bnn_pynq.models import model_with_cfg
from brevitas_examples.speech_to_text.quartznet import model_with_cfg
from finn.transformation.infer_shapes import InferShapes

#REF_MODEL = 'CNV_2W2A'
REF_MODEL = 'quant_quartznet_perchannelscaling_4b'

def test_debug_finn_onnx_export():
    model, cfg = model_with_cfg(REF_MODEL, pretrained=False, export_mode=True)
    model.eval()
    debug_hook = enable_debug(model)
    #input_tensor = torch.randn(1, 3, 32, 32)
    #inp_val = np.load("/workspace/results/librispeech_data/input_sample_float_0.npy")
    inp_val = np.load("input_sample_float_0.npy")
    print(inp_val.shape)
    input_tensor = torch.from_numpy(inp_val)
    export_finn_onnx(model, input_shape=input_tensor.shape, export_path='debug.onnx')
    model(input_tensor)
    assert debug_hook.values

    #model_onnx = ModelWrapper("/workspace/brevitas/tests/brevitas_finn/brevitas/debug.onnx")
    #model_onnx = ModelWrapper("debug.onnx")
    #print(len(model_onnx.graph.node))
    # Try first repartitioning...
    #model_onnx = model_onnx.transform(InferShapes(), make_deepcopy=False)
    #input_dict = {model_onnx.graph.input[0].name: inp_val}
    #output_dict = oxe.execute_onnx(model_onnx, input_dict, return_full_exec_context=True)
    
    #return output_dict, debug_hook.values
    
    return debug_hook.values


In [3]:
#output_dict, debughook = test_debug_finn_onnx_export()
debughook = test_debug_finn_onnx_export()

(1, 64, 256)


/workspace/brevitas/src/brevitas/quant_tensor/__init__.py:74: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  training = torch.tensor(training, dtype=torch.bool)


In [4]:
from finn.util.visualization import showInNetron
#showInNetron("debug.onnx")
showInNetron("/tmp/quartznet_export.onnx")

Serving '/tmp/quartznet_export.onnx' at http://0.0.0.0:8081


In [5]:
model = ModelWrapper("/tmp/quartznet_export.onnx")

inp_val = np.load("input_sample_float_0.npy")
input_dict = {model.graph.input[0].name: inp_val}
output_dict = oxe.execute_onnx(model, input_dict, return_full_exec_context=True)



In [9]:
print(debughook)

{'encoder.encoder.0.mconv.0.input': tensor([[[-1.9984, -0.6098, -0.5618,  ...,  0.2448,  0.5254, -0.4095],
         [-1.8404, -0.5956, -0.5880,  ..., -0.1198, -0.5046, -1.0198],
         [-1.0554, -0.6672, -0.8453,  ..., -0.6674, -0.8695, -0.8649],
         ...,
         [-1.1238, -0.8380, -1.0038,  ..., -0.9721, -0.9777, -0.6696],
         [-1.2350, -1.1575, -1.0161,  ..., -1.0645, -0.9239, -0.8753],
         [-0.8929, -1.0764, -0.9956,  ..., -0.9764, -0.8264, -1.0060]]]), 'encoder.encoder.0.mconv.0.output': tensor([[[-0.1382, -0.1806, -0.2267,  ...,  0.0825,  0.1692,  0.0275],
         [-0.1706, -0.1198, -0.2315,  ..., -0.0254, -0.0956, -0.0102],
         [-0.0158,  0.0212, -0.0065,  ...,  0.1979,  0.2124,  0.2467],
         ...,
         [-0.0081,  0.0024,  0.0979,  ...,  0.0818,  0.0395,  0.0170],
         [ 0.0727,  0.0419,  0.0485,  ..., -0.1439, -0.1188, -0.1525],
         [ 0.1615,  0.1869,  0.1647,  ...,  0.0672,  0.0749,  0.1039]]]), 'encoder.encoder.0.norm_depthwise.0.input'

In [8]:
print(output_dict)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [25]:
for k in debughook.keys():
    if k=="encoder.encoder.0.mconv.0.input":
        onnx_val = output_dict["inp.1"]
        brevitas_val = debughook[k]
        print(np.isclose(onnx_val, brevitas_val, atol=1e-3).all())
    if k=="encoder.encoder.0.mconv.0.output":
        onnx_val = output_dict["811"]
        brevitas_val = debughook[k].numpy()
        print(np.isclose(onnx_val, brevitas_val, atol=1e-3).all())
        print(onnx_val.flatten()-brevitas_val.flatten())
    if k=="encoder.encoder.0.norm_depthwise.0.output":
        onnx_val = output_dict["817"]
        brevitas_val = debughook[k].numpy()
        print(np.isclose(onnx_val, brevitas_val, atol=1e-3).all())
        print(onnx_val.flatten()-brevitas_val.flatten())
    if k=="encoder.encoder.0.mconv.1.output":
        onnx_val = output_dict["821"]
        brevitas_val = debughook[k].numpy()
        print(np.isclose(onnx_val, brevitas_val, atol=1e-3).all())
        print(onnx_val.flatten()-brevitas_val.flatten())
    if k=="encoder.encoder.0.mout.0.input":
        onnx_val = output_dict["822"]
        brevitas_val = debughook[k].numpy()
        print(np.isclose(onnx_val, brevitas_val, atol=1e-3).all())
        print(onnx_val.flatten()-brevitas_val.flatten())
    if k=="encoder.encoder.0.mout.0.output":
        onnx_val = output_dict["826"]
        brevitas_val = debughook[k].numpy()
        print(np.isclose(onnx_val, brevitas_val, atol=1e-3).all())
        print(onnx_val.flatten()-brevitas_val.flatten())
    if k=="encoder.encoder.1.res.0.0.input":
        onnx_val = output_dict["826"]
        brevitas_val = debughook[k].numpy()
        print(np.isclose(onnx_val, brevitas_val, atol=1e-3).all())
        print(onnx_val.flatten()-brevitas_val.flatten())
    if k=="encoder.encoder.1.res.0.0.output":
        onnx_val = output_dict["927"]
        brevitas_val = debughook[k]
        print(np.isclose(onnx_val, brevitas_val, atol=1e-3).all())
    if k=="encoder.encoder.1.quant_normalization.input":
        onnx_val = output_dict["928"]
        brevitas_val = debughook[k]
        print(np.isclose(onnx_val, brevitas_val, atol=1e-3).all())
    if k=="encoder.encoder.1.quant_normalization.output":
        onnx_val = output_dict["931"]
        brevitas_val = debughook[k]
        print(np.isclose(onnx_val, brevitas_val, atol=1e-3).all())
    if k=="encoder.encoder.1.mconv.0.output":
        onnx_val = output_dict["830"]
        brevitas_val = debughook[k]
        print(np.isclose(onnx_val, brevitas_val, atol=1e-3).all())
    if k=="encoder.encoder.1.norm_depthwise.0.output":
        onnx_val = output_dict["836"]
        brevitas_val = debughook[k]
        print(np.isclose(onnx_val, brevitas_val, atol=1e-3).all())
    if k=="encoder.encoder.1.mconv.1.output":
        onnx_val = output_dict["840"]
        brevitas_val = debughook[k]
        print(np.isclose(onnx_val, brevitas_val, atol=1e-3).all())
    if k=="encoder.encoder.1.mconv.3.input":
        onnx_val = output_dict["841"]
        brevitas_val = debughook[k]
        print(np.isclose(onnx_val, brevitas_val, atol=1e-3).all())
    if k=="encoder.encoder.1.mconv.3.output":
        onnx_val = output_dict["845"]
        brevitas_val = debughook[k]
        print(np.isclose(onnx_val, brevitas_val, atol=1e-3).all())
    if k=="encoder.encoder.1.mconv.5.output":
        onnx_val = output_dict["849"]
        brevitas_val = debughook[k]
        print(np.isclose(onnx_val, brevitas_val, atol=1e-3).all())
    if k=="encoder.encoder.1.norm_depthwise.1.output":
        onnx_val = output_dict["855"]
        brevitas_val = debughook[k]
        print(np.isclose(onnx_val, brevitas_val, atol=1e-3).all())
    if k=="encoder.encoder.1.mconv.6.output":
        onnx_val = output_dict["859"]
        brevitas_val = debughook[k]
        print(np.isclose(onnx_val, brevitas_val, atol=1e-3).all())
    if k=="encoder.encoder.1.mconv.8.input":
        onnx_val = output_dict["860"]
        brevitas_val = debughook[k]
        print(np.isclose(onnx_val, brevitas_val, atol=1e-3).all())
    if k=="encoder.encoder.1.mconv.8.input":
        onnx_val = output_dict["864"]
        brevitas_val = debughook[k]
        print(np.isclose(onnx_val, brevitas_val, atol=1e-3).all())

True
False
[0.11371705 0.0706642  0.2348292  ... 0.10824388 0.11475077 0.09204154]
False
[0.11346879 0.07106266 0.23742498 ... 0.10348724 0.11196846 0.09396221]
False
[ 0.00122932  0.02612335 -0.03658096 ...  0.16832887  0.08221649
  0.13204402]
False
[0.19059996 0.22499031 0.16348241 ... 0.44359052 0.24687213 0.36536622]
False
[0.188156   0.19890776 0.16506796 ... 0.30642548 0.16127656 0.24729073]
False
False
False
False
False
False
False
False
False
False
False
False
False
[0.188156   0.19890776 0.16506796 ... 0.30642548 0.16127656 0.24729073]
False


In [1]:
!ls

# Note that we have the model debug.onnx

FCLayer_graph.onnx  advanced	end2end_example
LFCW1A1.onnx	    basics	input_sample_float_0.npy
QuartzNet	    debug.onnx	test_debug_export.ipynb


In [75]:
from finn.util.visualization import showInNetron

showInNetron("/workspace/brevitas/tests/brevitas_finn/brevitas/debug.onnx")

Stopping http://0.0.0.0:8081
Serving '/workspace/brevitas/tests/brevitas_finn/brevitas/debug.onnx' at http://0.0.0.0:8081


# Old results that I cannot reproduce...

In [79]:
debughook

{'encoder.encoder.0.mconv.0.input': tensor([[[-1.9984, -0.6098, -0.5618,  ...,  0.2448,  0.5254, -0.4095],
          [-1.8404, -0.5956, -0.5880,  ..., -0.1198, -0.5046, -1.0198],
          [-1.0554, -0.6672, -0.8453,  ..., -0.6674, -0.8695, -0.8649],
          ...,
          [-1.1238, -0.8380, -1.0038,  ..., -0.9721, -0.9777, -0.6696],
          [-1.2350, -1.1575, -1.0161,  ..., -1.0645, -0.9239, -0.8753],
          [-0.8929, -1.0764, -0.9956,  ..., -0.9764, -0.8264, -1.0060]]]),
 'encoder.encoder.0.mconv.0.output': tensor([[[ 0.0762,  0.0606, -0.0257,  ...,  0.0669,  0.0092,  0.0167],
          [ 0.1476,  0.2524,  0.1607,  ..., -0.0403, -0.1418, -0.1523],
          [-0.1454, -0.0765, -0.1340,  ..., -0.0420, -0.0282, -0.0431],
          ...,
          [-0.0678, -0.0147,  0.0461,  ..., -0.0935, -0.0944, -0.0715],
          [ 0.0489,  0.0197,  0.0235,  ..., -0.0904, -0.0731, -0.0625],
          [-0.1974, -0.2090, -0.1920,  ..., -0.1585, -0.1414, -0.1474]]]),
 'encoder.encoder.0.norm_dept

In [80]:
output_dict

{'inp.1': array([[[-1.9983572 , -0.60976577, -0.5618071 , ...,  0.244798  ,
           0.5254309 , -0.40952483],
         [-1.8404343 , -0.5956033 , -0.5879617 , ..., -0.11982724,
          -0.5045655 , -1.0197967 ],
         [-1.0554066 , -0.66715354, -0.84530157, ..., -0.66736484,
          -0.86954856, -0.8648965 ],
         ...,
         [-1.1237619 , -0.8379547 , -1.003774  , ..., -0.97209734,
          -0.9776587 , -0.669581  ],
         [-1.2349979 , -1.1574824 , -1.0160869 , ..., -1.0645214 ,
          -0.92387044, -0.87531835],
         [-0.89286107, -1.0764301 , -0.99561274, ..., -0.9763662 ,
          -0.82644486, -1.0059506 ]]], dtype=float32),
 '3225': array([[20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20,
         20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20,
         20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20,
         20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20,
         20, 20, 20, 20, 20, 20, 20

In [81]:
from finn.custom_op.registry import getCustomOp

diff = {}
for n in model.graph.node:
    if n.op_type=="DebugMarker":
        inst = getCustomOp(n)
        debug_key = inst.get_nodeattr("export_debug_name")
        dict_key = n.input[0]
        debug_val = debughook[debug_key]
        dict_val = output_dict[dict_key]
        
        assert(debug_val.shape==dict_val.shape), "Different shapes detected!"
        
        is_close = np.isclose(debug_val, dict_val, atol=1e-3).all()
        print(is_close)
        
        if not is_close:
            difference = debug_val.numpy().flatten()-dict_val.flatten()
            diff[dict_key] = np.abs(difference)



True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
False
True
True
True
True
True
True
True
True
True
True
True
True
Tr

In [84]:
for k,v in diff.items():
    print("{}\t{}".format(k,v))

963	[0.08397286 0.05537734 0.06456659 ... 0.03313809 0.07704154 0.05484924]
970	[0.125 0.    0.125 ... 0.    0.125 0.   ]
1119	[0.0430885  0.05129582 0.03282933 ... 0.07616858 0.10087193 0.01646888]
1126	[0.    0.    0.    ... 0.125 0.125 0.   ]
1275	[0.00822909 0.08434818 0.07200454 ... 0.04302304 0.01843845 0.00409743]
1282	[0.    0.125 0.125 ... 0.    0.    0.   ]
1431	[0.04519024 0.03902794 0.09243459 ... 0.02431407 0.05065433 0.02026173]
1438	[0.    0.    0.125 ... 0.    0.    0.   ]
1587	[0.03908042 0.01645492 0.02879611 ... 0.00617911 0.08444781 0.        ]
1594	[0.    0.    0.    ... 0.    0.125 0.   ]
1743	[0.00821354 0.02053386 0.02464063 ... 0.00618254 0.         0.        ]
1750	[0. 0. 0. ... 0. 0. 0.]
1899	[0.02349504 0.01006931 0.01846039 ... 0.         0.         0.        ]
1906	[0. 0. 0. ... 0. 0. 0.]
2055	[0.         0.0101884  0.00436646 ... 0.         0.         0.        ]
2062	[0. 0. 0. ... 0. 0. 0.]
2211	[0. 0. 0. ... 0. 0. 0.]
2218	[0. 0. 0. ... 0. 0. 0.]
2367	[

In [86]:
print(np.array_equal(debughook["decoder.decoder_layers.0.output"], output_dict["3221"]))

True
